In [ ]:
import requests
import urllib3
import json
import csv
import os
import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
pd.set_option('display.max_columns', 83)
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.max_rows', None)
pd.set_option("display.precision", 2)
sns.set(rc={'figure.figsize':(12.7,8.27)})

In [ ]:
cd = os.path.abspath(os.getcwd())
with open(f'{cd}/config.json') as f:
    data = json.load(f)

### getting activities from strava api

In [ ]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"
payload = data['payload']
res = requests.post(auth_url, data=payload, verify=False)
header = {'Authorization': 'Bearer ' + res.json()['access_token']}

my_activities = pd.DataFrame()
for page in range(1,7):
    my_dataset = requests.get(activites_url, headers=header, params={'per_page': 200, 'page': page}).json()
    my_dataframe = pd.json_normalize(my_dataset)
    my_activities = pd.concat([my_activities, my_dataframe], ignore_index=True)

In [ ]:
my_activities.shape

##### write raw activities/get only runs and walks, remove NaNs/write them also

In [ ]:
with open(f'{cd}/csvs/raw/all_activities_raw.csv', 'w') as f:
    my_activities.to_csv(f)

In [ ]:
runs_maps = my_activities[my_activities['type'] == 'Run'][['id', 'map.summary_polyline']]
walks_maps = my_activities[my_activities['type'] == 'Walk'][['id', 'map.summary_polyline']]
runs_maps = runs_maps.dropna()
walks_maps = walks_maps.dropna()

In [ ]:
with open(f'{cd}/csvs/clean/runs.csv', 'w') as runs_file, open(f'{cd}/csvs/clean/walks.csv', 'w') as walks_file:
    runs_maps.to_csv(runs_file)
    walks_maps.to_csv(walks_file)

##### create a deep copy of my_activities and do things to it

In [ ]:
my_acts = my_activities.copy(deep=True)

In [ ]:
my_acts = my_acts.rename(columns={'average_speed' : 'average_speed_mps', 
                                        'max_speed' : 'max_speed_mps',
                                        'moving_time' : 'moving_time_s',
                                        'elapsed_time' : 'elapsed_time_s'})

# fix 'start_date_local' and create 2 new columns 'start_time' and 'moving_time(min)'
my_acts['start_date_local'] = pd.to_datetime(my_acts['start_date_local'])
my_acts['start_time'] = my_acts['start_date_local'].dt.time
my_acts['start_date_local'] = my_acts['start_date_local'].dt.strftime("%Y/%m/%d")
my_acts['start_date_local'] = pd.to_datetime(my_acts['start_date_local'])
my_acts['start_day_name'] = my_acts['start_date_local'].dt.strftime("%A")


# seconds to minutes
my_acts['moving_time(min)'] = my_acts['moving_time_s'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%H:%M:%S'))

# meters to kms
my_acts['distance'] = my_acts[
    (my_acts['type'] == 'Run') |
    (my_acts['type'] == 'Walk')]['distance'] / 1000

# meters per second to kms per hour
my_acts['average_speed_mps'] = my_acts['average_speed_mps'] * (18/5)
my_acts['max_speed_mps'] = my_acts['max_speed_mps'] * (18/5)

# from (GMT+01:00) Europe/Berlin -> Europe/Berlin
my_acts['timezone'] = my_acts['timezone'].str.split(' ').str[-1]

# rename converted columns
my_acts = my_acts.rename(columns={'average_speed_mps': 'average_speed_kmh',
                                        'max_speed_mps' : 'max_speed_kmh',
                                        'distance' : 'distance_km'})

##### leave only needed columns

In [ ]:
cols = ['upload_id', 'name', 'type', 'distance_km', 'moving_time(min)','start_time',  'start_date_local', 'start_day_name', 
        'timezone','average_speed_kmh', 'max_speed_kmh', 'total_elevation_gain', 'average_heartrate',
        'max_heartrate', 'achievement_count', 'kudos_count', 'visibility',  ]
my_acts = my_acts[cols]

# usage -> 25.1 KB to 22.6 KB
my_acts['visibility'] = pd.Categorical(my_acts['visibility'])
my_acts['timezone'] = pd.Categorical(my_acts['timezone'])

##### save cleaned activities

In [ ]:
with open(f'{cd}/csvs/clean/all_activities.csv', 'w') as activs_f:
    my_acts.to_csv(activs_f)

In [ ]:
with open(f'{cd}/csvs/clean/all_activities.csv', 'r') as activs_f:
    activities = pd.read_csv(activs_f, delimiter=',' , encoding='iso-8859-1', index_col=[0])

In [ ]:
run = activities.loc[activities['type'] == 'Run']
walk = activities.loc[activities['type'] == 'Walk']
run_5_more = run[run['distance_km'] > 5]

#### Countplots

In [ ]:
sns.countplot(x='type',
              data=activities)

In [ ]:
sns.countplot(x='start_day_name', data=run)

In [ ]:
sns.countplot(x='start_day_name', data=walk)

In [ ]:
sns.countplot(x='name', 
              data=run, 
              order=pd.value_counts(run['name']).iloc[:4].index)
locs, labels = plt.xticks(rotation=45, fontsize=8)
plt.title("Different named runs and their couns")
plt.xlabel("Name of run")
plt.ylabel("Count")

#### Relplots

In [ ]:
sns.relplot(x='start_day_name', y='distance_km', hue='name',
              data=run_5_more
           )
locs, labels = plt.xticks(rotation=45, fontsize=8)
plt.title("Different days of week and distances")
plt.xlabel("Days")
plt.ylabel("Distance")

In [ ]:
# Find max for each day
sns.relplot(x='start_day_name', y='distance_km', 
            kind='line', ci=None,
            data=run)
locs, labels = plt.xticks(rotation=45, fontsize=15)
plt.title("Different days of week and distances")
plt.xlabel("Days")
plt.ylabel("Distance")

In [ ]:
sns.relplot(x='distance_km', y='moving_time(min)', hue='start_day_name',
            col='start_day_name', col_wrap=3, data=run)

#### Pointplots

In [ ]:
sns.pointplot(x='start_day_name', y='distance_km',  data=run_5_more)
plt.title("Different days of week and distances 5km and more")
plt.xlabel("Days")
plt.ylabel("Distance")

#### Violinplots

In [ ]:
sns.violinplot(x='start_day_name', y='distance_km', palette='muted', data=run)
plt.title("Different days of week and distances")
plt.xlabel("Days")
plt.ylabel("Distance")

#### Scatterplots + FacetGrid

In [ ]:
sns.scatterplot(x='distance_km',
                y='moving_time(min)',
 b               data=run_5_more)
locs,label = plt.xticks(rotation=90, fontsize=3)

In [ ]:
g = sns.FacetGrid(run, col='start_day_name', col_wrap=2)
g = g.map(sns.scatterplot, 'distance_km', 'max_speed_kmh')
#plt.title("Different days of week and distances")


#### Regplots

In [ ]:
sns.set(style="ticks", context="talk")
sns.regplot(x="distance_km", y="average_speed_kmh", data=run).set_title("Average Speed vs Distance")

In [ ]:
sns.set(style="ticks", context="talk")
sns.regplot(x='distance_km', y='max_speed_kmh', data=run).set_title("Max Speed vs Distance")

In [ ]:
sns.boxplot(x='name',y='distance_km',data=run_5_more)
locs, labels = plt.xticks(rotation=45, fontsize=12)

#### Distplots

In [ ]:
sns.displot(run['distance_km'])